Imports de codigo

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import sys
import os
from tqdm import tqdm
import time
import http.client
import json
import pandas as pd
src_path = os.path.abspath(os.path.join(os.pardir, 'src'))
sys.path.append(src_path)
import os
import dotenv as dotenv

Datos a introducir

In [20]:
dotenv.load_dotenv()
api_key=os.getenv("apikey_booking")
key = "d66a7bdef8mshdb55378189405fbp10a9ffjsnd9d65bafe8b6"
# Fecha de búsqueda
Ciudad="Roma"


Codigo de extraccion de API Booking

[Documentación API Booking](https://developers.booking.com/)

In [21]:
import requests

url = "https://booking-com.p.rapidapi.com/v1/hotels/locations"

querystring = {"locale":"en-gb","name":Ciudad}

headers = {
	"x-rapidapi-key": key,
	"x-rapidapi-host": "booking-com.p.rapidapi.com"
}

response_Id= requests.get(url, headers=headers, params=querystring)

print(response_Id.json())

[{'nr_hotels': 26013, 'city_ufi': None, 'dest_type': 'country', 'city_name': '', 'lc': 'en', 'name': 'Romania', 'cc1': 'ro', 'country': 'Romania', 'label': 'Romania', 'longitude': 24.985567, 'image_url': 'https://cf.bstatic.com/xdata/images/country/150x150/1196.jpg?k=07d66f78280bfe5c92d14dc52c570c057afe14a196b5902cac363f0239b71a3e&o=', 'b_max_los_data': {'is_fullon': 0, 'default_los': 45, 'max_allowed_los': 90, 'extended_los': 90, 'has_extended_los': 1, 'experiment': 'long_stays_android_extend_los_2'}, 'roundtrip': 'GgEwIAAoATICZW46BFJvbWFAAEoAUAA=', 'hotels': 26013, 'latitude': 45.839714, 'rtl': 0, 'type': 'co', 'dest_id': '175', 'region': ''}, {'dest_id': '1089', 'type': 'la', 'rtl': 0, 'region': 'Lazio', 'hotels': 733, 'latitude': 41.901005, 'cc1': 'it', 'label': 'Roma Termini, Rome, Lazio, Italy', 'country': 'Italy', 'longitude': 12.501904, 'image_url': 'https://cf.bstatic.com/xdata/images/landmark/150x150/173174.jpg?k=3468b8d784c996306802c3889b115b83438b955ffdf982dbc8030ec116a7995

In [18]:
# dest_id': '-1456928' Paris
#  dest_id': '-126693' Roma

# Inicializar un DataFrame final
df_final = pd.DataFrame()

# Extraer datos de entrada
adults_number = 2
checkin_date = '2025-2-13'
checkout_date = '2025-2-16'
dest_id = '-126693'
checkin_date = '2025-2-13'
destination = 'Roma'

# Realizar la solicitud a la API
dicc_datos = dict()
try:
    conn = http.client.HTTPSConnection("booking-com.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': key,
        'x-rapidapi-host': "booking-com.p.rapidapi.com"
    }
    url = f"/v1/hotels/search?page_number=0&adults_number={adults_number}&room_number=1&include_adjacency=true&units=metric&checkout_date={checkout_date}&dest_id={dest_id}&filter_by_currency=EUR&dest_type=city&checkin_date={checkin_date}&order_by=popularity&locale=es"
    
    conn.request("GET", url, headers=headers)
    res = conn.getresponse()
    data = res.read()
    dicc_datos = json.loads(data.decode("utf-8"))

except Exception as e:
    print(f"Error al hacer la petición a la API: {e}")

# Procesar los datos obtenidos
rows = []
try:
    hoteles = dicc_datos.get('result', [])
    for h in hoteles:
        row = {
            'destination': destination,
            'city': h.get('city'),
            'checkin_date': checkin_date,
            'checkout_date': checkout_date,
            'address': h.get('address'),
            'accommodation_type_name': h.get('accommodation_type_name'),
            'distance': h.get('distance'),
            'checkin': h.get('checkin', {}).get('from'),
            'checkout': h.get('checkout', {}).get('until'),
            'review_score': h.get('review_score'),
            'price_total': h.get('price_breakdown', {}).get('all_inclusive_price'),
            'price_night': h.get('composite_price_breakdown', {}).get('gross_amount_per_night', {}).get('value'),
            'hotel_name': h.get('hotel_name'),
            'class': h.get('class')
        }
        rows.append(row)

except Exception as e:
    print(f"Error al recorrer datos devueltos: {e}")

# Agregar los resultados a un DataFrame temporal
df_temp = pd.DataFrame(rows)
df_final = pd.concat([df_final, df_temp], ignore_index=True)

# Mostrar el DataFrame final
print(df_final)


   destination  city checkin_date checkout_date  \
0         Roma  Roma    2025-2-13     2025-2-16   
1         Roma  Roma    2025-2-13     2025-2-16   
2         Roma  Roma    2025-2-13     2025-2-16   
3         Roma  Roma    2025-2-13     2025-2-16   
4         Roma  Roma    2025-2-13     2025-2-16   
5         Roma  Rome    2025-2-13     2025-2-16   
6         Roma  Roma    2025-2-13     2025-2-16   
7         Roma  Roma    2025-2-13     2025-2-16   
8         Roma  Roma    2025-2-13     2025-2-16   
9         Roma  Roma    2025-2-13     2025-2-16   
10        Roma  Roma    2025-2-13     2025-2-16   
11        Roma  Rome    2025-2-13     2025-2-16   
12        Roma  Roma    2025-2-13     2025-2-16   
13        Roma  Roma    2025-2-13     2025-2-16   
14        Roma  Roma    2025-2-13     2025-2-16   
15        Roma  Roma    2025-2-13     2025-2-16   
16        Roma  Roma    2025-2-13     2025-2-16   
17        Roma  Roma    2025-2-13     2025-2-16   
18        Roma  Roma    2025-2-

In [19]:
# Exportación del DataFrame a un archivo CSV
df_final.to_csv(f'../data/alojamientos{Ciudad}.csv', index=False)